In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import time

#Conversions

# 1 year = 31536000 s
# 1 pc = 3.086e16 m
# 1 M_sun = 1.989e30 kg

#Some Constants Used Throughout

t_obsy = 4                  #years
t_obs = t_obsy*31536000     #s
v_t = 6.481*10**(-12)       #pc/s
d_l = 50                    #pc
d_s = 2000                  #pc
sigv = 3*10**(-28)          #cm^3/s
m_chi = 100                 #GeV
pi = 3.141592653

Smin = []                   #The four cutoffs of Smin, in microarcseconds
Smin.append(4)
Smin.append(16)
Smin.append(64)
Smin.append(256)

#Some Parameters

N = 82                      #Number of Epochs
rho_0 = 2.0*10**8           #Maximum core density from dark matter annihilation (M_sun/pc^3)
m_trunc = 100               #Parameter dictating truncation radius
m_core = 1                  #Parameter dictating core radius

#Some Functions Used Throughout

def r_t(m):                                        #Truncation Radius in pc
    return (0.019*(1000/1)*(m*(1+3250))**(1/3))

def r_c(m):                                        #Core Radius in pc
    return (3.3*10**(-4)*m**(1/3))

def S(b, p):                                       #Signal in as
    return ((N**(1/2))*((b**2 + p**2)**(1/2)))

def rho(r,m):                                      #Density in M_sun/pc^3
    if(r < r_t(m)):
        return (rho_0*(1+r/(r_c(m)))**(-9/4))
    else:
        return 0
    
def A_l(by, bx):                                   #Lensed Area in as^2
    return (by*bx)

def M2D(xi, z):                                    #M2D function
    return 4*pi*rho_0*(1+(np.sqrt((xi**2) + (z**2))/r_c(m_core)))**(-9/4)*xi

def func(xi):                                      #Boundary function in M2D integral
    return np.sqrt(r_t(m_trunc)**2 - xi**2)

def mass_int(xip):
    return scipy.integrate.dblquad(M2D, 0, xip, 0, func)

def beta_mag(by, bx):
    return np.sqrt(bx**2 + by**2)

In [2]:
beta_twid = np.linspace(0, 100.0, 1001)
phi = np.linspace(0, 1.0, 11)

beta_twid_pass = []
phi_pass = []

beta_y = v_t*t_obs/d_l*beta_twid
beta_x0 = v_t*t_obs/d_l*phi

mass = []
area = []

for i in range(np.size(beta_twid)):
    for j in range(np.size(phi)):
        if (S(beta_y[i],beta_x0[j]) > Smin[3]*10**(-6)):
            beta_twid_pass.append(beta_twid[i])
            phi_pass.append(phi[j])
            
beta_y_pass = [v_t*t_obs/d_l*x for x in beta_twid_pass]
beta_x0_pass = [v_t*t_obs/d_l*y for y in phi_pass]
            
#for i in range(np.size(beta_twid_pass)):
#    for j in range(np.size(phi_pass)):
#        area.append(A_l(beta_y_pass[i],beta_x0_pass[j]))
#        
#        xip = d_l*beta_mag(beta_y_pass[i], beta_x0_pass[j])
#        if  xip < r_t(m_trunc):
#            value, error = mass_int(xip)
#            mass.append(value)
            
#    if i%10 == 0:
#        print( i / np.size(beta_twid_pass))

In [3]:
a = d_l/(v_t*t_obs)

np.size(beta_x0_pass)

10826

In [4]:
start = time.time()
scipy.integrate.dblquad(M2D, 0, 4, 0, func)
end = time.time()
print(end-start)

/home/tommy/anaconda3/lib/python3.6/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


13.137790441513062


/home/tommy/anaconda3/lib/python3.6/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)


The integral takes a while to run, now up to ~13 seconds from the previous ~9 seconds. I'm also not really liking the associated errors. I might just have a bad computer, definitiely a possibility.